In [1]:
# Import python packages
import numpy as np
import pandas as pd
from fuzzywuzzy import process as fwprocess
import rpy2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

In [2]:
# Can uncomment to install R packages as needed
#utils = importr('utils')
#utils.install_packages('nflreadr')
#utils.install_packages('ffscrapr')

In [3]:
# Import R packages
nflreadr = importr('nflreadr')
ffscrapr = importr('ffscrapr')

In [4]:
# Config for the calculation
mfl_id = 60206  #MFL id for league
seasons = [2023,2022,2021]  #Seasons to collect data for
num_off = 7  #Number of offensive players to include in calculation for each game
num_def = 12  #Number of defensive  players to include in calculation for each game
outfile = 'Analytics_Dynasty_League_Lineups_2024.csv'  #Where to save results

In [5]:
# A mapping from PFR team names to MFL team names

###
team_dic = {
    'ARI' : 'ARI',
    'ATL' : 'ATL',
    'BAL' : 'BAL',
    'BUF' : 'BUF',
    'CAR' : 'CAR',
    'CHI' : 'CHI',
    'CIN' : 'CIN',
    'CLE' : 'CLE',
    'DAL' : 'DAL',
    'DEN' : 'DEN',
    'DET' : 'DET',
    'GB' : 'GBP',
    'HOU' : 'HOU',
    'IND' : 'IND',
    'JAX' : 'JAC',
    'KC' : 'KCC',
    'LA' : 'LAR',
    'LAC' : 'LAC',
    'LV' : 'LVR',
    'MIA' : 'MIA',
    'MIN' : 'MIN',
    'NE' : 'NEP',
    'NO' : 'NOS',
    'NYG' : 'NYG',
    'NYJ' : 'NYJ',
    'PHI' : 'PHI',
    'PIT' : 'PIT',
    'SEA' : 'SEA',
    'SF' : 'SFO',
    'TB' : 'TBB',
    'TEN' : 'TEN',
    'WAS' : 'WAS',
}

In [6]:
# Mappings for positions, both to consider which positions to drop from the analysis and to cleanup PFR positions a bit

###
snap_side_map = {
    'C' : 'DROP',
    'CB' : 'DEF',
    'DB' : 'DEF',
    'DE' : 'DEF',
    'DT' : 'DEF',
    'FB' : 'OFF',
    'FS' : 'DEF',
    'G' : 'DROP',
    'K' : 'DROP',
    'LB' : 'DEF',
    'LS' : 'DROP',
    'NT' : 'DEF',
    'P' : 'DROP',
    'QB' : 'OFF',
    'RB' : 'OFF',
    'SS' : 'DEF',
    'T' : 'DROP',
    'TE' : 'OFF',
    'WR' : 'OFF',
}

snap_position_map = {
    'CB' : 'CB',
    'DB' : 'S',
    'DE' : 'DE',
    'DT' : 'DT',
    'FB' : 'RB',
    'FS' : 'S',
    'LB' : 'LB',
    'NT' : 'DT',
    'QB' : 'QB',
    'RB' : 'RB',
    'SS' : 'S',
    'TE' : 'TE',
    'WR' : 'WR',
}

###
pos_side_map = {
    'CB' : 'DEF',
    'Coach' : 'DROP',
    'DE' : 'DEF',
    'DT' : 'DEF',
    'Def' : 'DROP',
    'LB' : 'DEF',
    'Off' : 'DROP',
    'PK' : 'DROP',
    'PN' : 'DROP',
    'QB' : 'OFF',
    'RB' : 'OFF',
    'S' : 'DEF',
    'ST' : 'DROP',
    'TE' : 'OFF',
    'TMDB' : 'DROP',
    'TMDL' : 'DROP',
    'TMLB' : 'DROP',
    'TMPK' : 'DROP',
    'TMPN' : 'DROP',
    'TMQB' : 'DROP',
    'TMRB' : 'DROP',
    'TMTE' : 'DROP',
    'TMWR' : 'DROP',
    'WR' : 'OFF',
    'XX' : 'DROP'}

In [7]:
# Scrape snap counts from PFR and convert to pandas df
snap_df_r = nflreadr.load_snap_counts(ro.IntVector(seasons))

with (ro.default_converter + pandas2ri.converter).context():
  snap_df = ro.conversion.get_conversion().rpy2py(snap_df_r)

# Cleanup some of the data
snap_df['team'] = snap_df['team'].apply(lambda x: team_dic[x])
snap_df['side'] = snap_df['position'].apply(lambda x: snap_side_map[x])

snap_df = snap_df[ snap_df['side'] != 'DROP' ]

snap_df['position'] = snap_df['position'].apply(lambda x: snap_position_map[x])
snap_df['player'] = snap_df['player'].apply(lambda x: nflreadr.clean_player_names(x, convert_lastfirst=False)[0])
snap_df['join_string'] = snap_df['player'] + " " + snap_df['team'] + " " + snap_df['position']

snap_df

Note: nflreadr caches (i.e., stores a saved version) data by default.
If you expect different output try one of the following:
ℹ Restart your R Session or
ℹ Run `nflreadr::.clear_cache()`.
To disable this warning, run `options(nflreadr.verbose = FALSE)` or add it to your .Rprofile
This message is displayed once every 8 hours.


,game_id,pfr_game_id,season,game_type,week,player,pfr_player_id,position,team,opponent,offense_snaps,offense_pct,defense_snaps,defense_pct,st_snaps,st_pct,side,join_string
4,2023_01_ARI_WAS,202309100was,2023,REG,1,Sam Howell,HoweSa00,QB,WAS,ARI,71.0,1.00,0.0,0.0,0.0,0.00,OFF,Sam Howell WAS QB
7,2023_01_ARI_WAS,202309100was,2023,REG,1,Terry McLaurin,McLaTe00,WR,WAS,ARI,63.0,0.89,0.0,0.0,0.0,0.00,OFF,Terry McLaurin WAS WR
8,2023_01_ARI_WAS,202309100was,2023,REG,1,Jahan Dotson,DotsJa00,WR,WAS,ARI,62.0,0.87,0.0,0.0,0.0,0.00,OFF,Jahan Dotson WAS WR
9,2023_01_ARI_WAS,202309100was,2023,REG,1,Logan Thomas,ThomLo00,TE,WAS,ARI,58.0,0.82,0.0,0.0,0.0,0.00,OFF,Logan Thomas WAS TE
10,2023_01_ARI_WAS,202309100was,2023,REG,1,Curtis Samuel,SamuCu00,WR,WAS,ARI,46.0,0.65,0.0,0.0,0.0,0.00,OFF,Curtis Samuel WAS WR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79300,2021_22_LA_CIN,202202130cin,2021,SB,22,Christian Rozeboom,RozeCh00,LB,LAR,CIN,0.0,0.00,0.0,0.0,22.0,0.73,DEF,Christian Rozeboom LAR LB
79301,2021_22_LA_CIN,202202130cin,2021,SB,22,Jake Gervase,GervJa00,S,LAR,CIN,0.0,0.00,0.0,0.0,22.0,0.73,DEF,Jake Gervase LAR S
79302,2021_22_LA_CIN,202202130cin,2021,SB,22,Travin Howard,HowaTr00,LB,LAR,CIN,0.0,0.00,0.0,0.0,22.0,0.73,DEF,Travin Howard LAR LB
79303,2021_22_LA_CIN,202202130cin,2021,SB,22,Grant Haley,HaleGr00,CB,LAR,CIN,0.0,0.00,0.0,0.0,22.0,0.73,DEF,Grant Haley LAR CB


In [8]:
# Dict to store data for each season
pos_df = {}

for s in seasons:
    # Scrape player positions from MFL and convert to pandas df
    mfl = ffscrapr.mfl_connect(season=s, league_id=mfl_id)
    
    pos_df_r = ffscrapr.mfl_players(mfl)
    
    with (ro.default_converter + pandas2ri.converter).context():
      pos_df[s] = ro.conversion.get_conversion().rpy2py(pos_df_r)

    # Cleanup some of the data
    pos_df[s]['season'] = s
    pos_df[s]['side'] = pos_df[s]['pos'].apply(lambda x: pos_side_map[x])

    pos_df[s] = pos_df[s][ pos_df[s]['side'] != 'DROP' ]
    
    pos_df[s]['player'] = pos_df[s]['player_name'].apply(lambda x: nflreadr.clean_player_names(x, convert_lastfirst=True)[0])
    pos_df[s]['join_string'] = pos_df[s]['player'] + " " + pos_df[s]['team'] + " " + pos_df[s]['pos']

#Merge seasons to single df
pos_df = pd.concat(pos_df, ignore_index=True)

pos_df

,player_id,player_name,pos,age,team,status,draft_year,draft_team,draft_pick,draft_round,...,twitter_username,height,jersey,college,weight,birthdate,season,side,player,join_string
0,4925,"Brees, Drew",QB,45.1,FA,NA_character_,2001,FA,1,2,...,drewbrees,72,9,Purdue,209,3301.0,2023,OFF,Drew Brees,Drew Brees FA QB
1,5848,"Brady, Tom",QB,46.6,FA,NA_character_,2000,NEP,33,6,...,NA_character_,76,12,Michigan,225,2771.0,2023,OFF,Tom Brady,Tom Brady FA QB
2,7401,"Roethlisberger, Ben",QB,42.0,FA,NA_character_,2004,PIT,11,1,...,NA_character_,77,7,Miami (Ohio),240,4443.0,2023,OFF,Ben Roethlisberger,Ben Roethlisberger FA QB
3,7836,"Rodgers, Aaron",QB,40.2,NYJ,NA_character_,2005,GBP,24,1,...,AaronRodgers12,74,8,California,223,5083.0,2023,OFF,Aaron Rodgers,Aaron Rodgers NYJ QB
4,8269,"Lewis, Marcedes",TE,39.8,CHI,NA_character_,2006,JAC,28,1,...,MarcedesLewis89,78,84,UCLA,265,5252.0,2023,OFF,Marcedes Lewis,Marcedes Lewis CHI TE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6902,15680,"Coyle, Tyler",S,25.3,DAL,R,2021,FA,NA_character_,NA_character_,...,NA_character_,73,31,Purdue,209,10545.0,2021,DEF,Tyler Coyle,Tyler Coyle DAL S
6903,15681,"Elliss, Christian",LB,25.1,PHI,R,2021,FA,NA_character_,NA_character_,...,NA_character_,75,53,Idaho,231,10593.0,2021,DEF,Christian Elliss,Christian Elliss PHI LB
6904,15682,"McCain, Mac",CB,26.0,PHI,R,2021,FA,NA_character_,NA_character_,...,NA_character_,72,29,North Carolina A&T,175,10263.0,2021,DEF,Mac McCain,Mac McCain PHI CB
6905,15683,"Saunders, C.J.",WR,27.4,CAR,R,2021,FA,NA_character_,NA_character_,...,NA_character_,70,81,Ohio State,190,9754.0,2021,OFF,CJ Saunders,CJ Saunders CAR WR


In [9]:
###
def GetClosestJoinString(other_df, this_js):
    # Return closest matching join string using fuzzy matching
    return fwprocess.extract(this_js, other_df['join_string'])[0][0]

def GetClosestJoinScore(other_df, this_js):
    # Return score of closest match using fuzzy matching
    return fwprocess.extract(this_js, other_df['join_string'])[0][1]

In [10]:
# Initialize merged df from snap data
df = snap_df[['game_id','season','week','pfr_player_id','player','team','position','side','join_string','offense_snaps','defense_snaps']].copy()

# Merge in positions using fuzzy string matching, considering each season and side of ball separately
df = df.set_index(['pfr_player_id','season','week']).sort_index(level=[0,1,2],ascending=[True,True,True])

for s in seasons:
    for side in ['OFF','DEF']:
        print(s,side)

        # Constrain data to this season and side of ball
        tmp_snap_df = snap_df[ (snap_df['season'] == s) & (snap_df['side'] == side) ]
        tmp_snap_df = tmp_snap_df.set_index(['pfr_player_id','season','week']).sort_index(level=[0,1,2],ascending=[True,True,True])
        tmp_snap_df = tmp_snap_df.groupby(['pfr_player_id','season']).tail(1).droplevel(2)

        tmp_pos_df = pos_df[ (pos_df['season'] == s) & (pos_df['side'] == side) ]

        # Fill the df with the closest matching join_string found by the fuzzy matching
        df.loc[ (df.index.get_level_values(1) == s) & (df['side'] == side), 'join_string'] = tmp_snap_df['join_string'].apply(lambda x: GetClosestJoinString(tmp_pos_df, x))

df = df.reset_index()

# With common join strings now in hand, we can merge in the position data from MFL
df = df.set_index(['join_string','season']).sort_index(level=[0,1],ascending=[True,True])
df = df.join( pos_df.set_index(['join_string','season']).sort_index(level=[0,1],ascending=[True,True])['pos'] )

df = df.reset_index()

# Rename the position columns
df = df.rename(columns={'position':'pfr_position'})
df = df.rename(columns={'pos':'position'})

# Find the per-game snap ranks for each player, assign qualifying players weight of 1, and break ties at the boundary with fractional weight
df = df.set_index(['game_id','team']).sort_index(level=[0,1],ascending=[True,True])

df['offense_rank'] = df.groupby(['game_id','team']).apply(lambda x: x['offense_snaps'].rank(method='first',ascending=False)).values  #Snap rank
df['offense_thresh'] = df.groupby(['game_id','team']).apply(lambda x: x['offense_snaps'][ x['offense_rank'] == num_off ].values[0])  #Snap boundary
df['offense_weight'] = ( (df['offense_rank'] <= num_off).values | (df['offense_snaps'] == df['offense_thresh']).values ).astype(float)  #Initial weights
thresh_inds = df['offense_snaps'] == df['offense_thresh']
df.loc[ thresh_inds, 'offense_weight' ] = df[thresh_inds].groupby(['game_id','team']).apply(lambda x: np.sum(x['offense_rank'] <= num_off) / len(x) )  #Normalize weights by breaking ties

df['defense_rank'] = df.groupby(['game_id','team']).apply(lambda x: x['defense_snaps'].rank(method='first',ascending=False)).values  #Snap rank
df['defense_thresh'] = df.groupby(['game_id','team']).apply(lambda x: x['defense_snaps'][ x['defense_rank'] == num_def ].values[0])  #Snap boundary
df['defense_weight'] = ( (df['defense_rank'] <= num_def).values | (df['defense_snaps'] == df['defense_thresh']).values ).astype(float)  #Initial weights
thresh_inds = df['defense_snaps'] == df['defense_thresh']
df.loc[ thresh_inds, 'defense_weight' ] = df[thresh_inds].groupby(['game_id','team']).apply(lambda x: np.sum(x['defense_rank'] <= num_def) / len(x) )  #Normalize weights by breaking ties

df = df.reset_index()

2023 OFF
2023 DEF
2022 OFF
2022 DEF
2021 OFF
2021 DEF


In [11]:
# Calculate the mean value for each season-team-position by summing over weights and dividing by number of games
df = df.set_index(['season','team','position']).sort_index(level=[0,1,2],ascending=[True,True,True])

result_off = df['offense_weight'].groupby(['season','team','position']).sum() / df.groupby(['season','team']).apply(lambda x: len(set(x['week'])))
result_off = result_off[ result_off >= 0.1 ]

result_def = df['defense_weight'].groupby(['season','team','position']).sum() / df.groupby(['season','team']).apply(lambda x: len(set(x['week'])))
result_def = result_def[ result_def >= 0.1 ]

df = df.reset_index()

# Pivot results out to columns to put into a nice tabular form
result_off = result_off.unstack(level=-1)
result_def = result_def.unstack(level=-1)

# Calculate values for DL and DB positions
result_def['DL'] = result_def['DT'] + result_def['DE']
result_def['DB'] = result_def['CB'] + result_def['S']

# Cleanup the table, listing the sorted values and season-teams for each position
result_dic = {}

for c in ['QB','RB','WR','TE']:
    tmp_df = result_off[c].reset_index()
    
    tmp_df[f'{c} Team'] = tmp_df['season'].map(str) + " " + tmp_df['team']
    tmp_df = tmp_df[[c, f'{c} Team']].rename(columns={c:f'{c} Value'})
    tmp_df = tmp_df.sort_values(f'{c} Value', ascending=False).reset_index(drop=True)

    result_dic[c] = tmp_df

for c in ['DT','DE','LB','CB','S','DL','DB']:
    tmp_df = result_def[c].reset_index()
    
    tmp_df[f'{c} Team'] = tmp_df['season'].map(str) + " " + tmp_df['team']
    tmp_df = tmp_df[[c, f'{c} Team']].rename(columns={c:f'{c} Value'})
    tmp_df = tmp_df.sort_values(f'{c} Value', ascending=False).reset_index(drop=True)

    result_dic[c] = tmp_df

# Save the table to a csv
result = pd.concat(result_dic, axis=1, ignore_index=False)
result.columns = result.columns.droplevel(0)

result.to_csv(outfile, index=False)

In [12]:
# Full results
result

,QB Value,QB Team,RB Value,RB Team,WR Value,WR Team,TE Value,TE Team,DT Value,DT Team,...,LB Value,LB Team,CB Value,CB Team,S Value,S Team,DL Value,DL Team,DB Value,DB Team
0,1.352941,2021 DEN,2.111111,2022 BAL,3.631579,2023 TBB,2.500000,2023 ATL,3.235294,2021 SEA,...,3.000000,2021 PIT,3.972222,2023 PIT,3.666667,2023 DAL,5.473684,2022 JAC,6.666667,2023 DAL
1,1.294118,2021 NOS,2.025000,2022 SFO,3.535088,2021 BUF,2.382353,2021 MIA,3.235294,2023 ATL,...,2.894737,2022 DAL,3.550000,2022 PHI,3.166667,2021 NEP,5.382353,2023 CAR,6.027778,2023 PIT
2,1.117647,2023 NOS,2.000000,2023 SFO,3.527778,2023 DAL,2.088235,2022 TEN,3.176471,2021 WAS,...,2.888889,2022 LAC,3.500000,2022 IND,3.055556,2022 BAL,5.333333,2022 MIA,5.842105,2022 DAL
3,1.117647,2021 HOU,1.975000,2021 SFO,3.516667,2021 KCC,2.088235,2022 NOS,3.176471,2022 ARI,...,2.882353,2022 ARI,3.456140,2023 TBB,3.000000,2023 MIN,5.323529,2021 JAC,5.647059,2022 NEP
4,1.058824,2023 CIN,1.911765,2021 BAL,3.500000,2022 CHI,2.029412,2021 LAC,3.166667,2023 CHI,...,2.789474,2023 BAL,3.416667,2022 BUF,2.950000,2023 DET,5.294118,2022 ATL,5.617647,2021 NOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,1.000000,2022 JAC,1.000000,2021 PIT,2.588235,2021 DEN,1.111111,2021 ARI,1.550000,2023 DET,...,1.735294,2023 CAR,2.294118,2022 ARI,1.647059,2022 IND,4.166667,2022 LAC,4.705882,2021 SEA
92,1.000000,2022 IND,1.000000,2023 LAC,2.549020,2022 ATL,1.111111,2023 MIA,1.539216,2021 NOS,...,1.722222,2023 LAR,2.250000,2022 SFO,1.588235,2021 HOU,4.166667,2021 PIT,4.592593,2022 MIA
93,1.000000,2022 HOU,1.000000,2023 JAC,2.382353,2021 MIA,1.105263,2023 BAL,1.500000,2021 SFO,...,1.666667,2021 LAR,2.205882,2023 MIN,1.561404,2023 TBB,4.055556,2023 DAL,4.588235,2022 ATL
94,1.000000,2022 GBP,1.000000,2023 CIN,2.205882,2023 ATL,1.088235,2021 BAL,1.444444,2023 DAL,...,1.558824,2023 MIN,2.150000,2023 DET,1.470588,2021 BAL,3.916667,2023 PIT,4.421053,2022 JAC


In [13]:
# Team 2
result.iloc[1]

QB Value    1.294118
QB Team     2021 NOS
RB Value       2.025
RB Team     2022 SFO
WR Value    3.535088
WR Team     2021 BUF
TE Value    2.382353
TE Team     2021 MIA
DT Value    3.235294
DT Team     2023 ATL
DE Value    3.141667
DE Team     2022 KCC
LB Value    2.894737
LB Team     2022 DAL
CB Value        3.55
CB Team     2022 PHI
S Value     3.166667
S Team      2021 NEP
DL Value    5.382353
DL Team     2023 CAR
DB Value    6.027778
DB Team     2023 PIT
Name: 1, dtype: object

In [14]:
# Team 48 (half of median)
result.iloc[47]

QB Value         1.0
QB Team     2021 CLE
RB Value    1.294118
RB Team     2023 DEN
WR Value    3.047619
WR Team     2021 CIN
TE Value    1.617647
TE Team     2023 WAS
DT Value    2.555556
DT Team     2021 LVR
DE Value    2.294118
DE Team     2022 NYJ
LB Value    2.047619
LB Team     2021 CIN
CB Value         2.9
CB Team     2021 KCC
S Value     2.117647
S Team      2021 CLE
DL Value    4.888889
DL Team     2021 TEN
DB Value    5.078431
DB Team     2023 ARI
Name: 47, dtype: object

In [15]:
# Team 49 (half of median)
result.iloc[48]

QB Value         1.0
QB Team     2023 LAR
RB Value    1.277778
RB Team     2021 PHI
WR Value    3.029412
WR Team     2022 IND
TE Value    1.611111
TE Team     2021 NEP
DT Value     2.54902
DT Team     2021 CLE
DE Value    2.277778
DE Team     2022 BUF
LB Value    2.029412
LB Team     2021 DEN
CB Value    2.898148
CB Team     2023 PHI
S Value     2.111111
S Team      2021 PHI
DL Value    4.882353
DL Team     2023 LVR
DB Value    5.058824
DB Team     2023 IND
Name: 48, dtype: object

In [16]:
# Team 95
result.iloc[94]

QB Value         1.0
QB Team     2022 GBP
RB Value         1.0
RB Team     2023 CIN
WR Value    2.205882
WR Team     2023 ATL
TE Value    1.088235
TE Team     2021 BAL
DT Value    1.444444
DT Team     2023 DAL
DE Value    1.411765
DE Team     2022 PIT
LB Value    1.558824
LB Team     2023 MIN
CB Value        2.15
CB Team     2023 DET
S Value     1.470588
S Team      2021 BAL
DL Value    3.916667
DL Team     2023 PIT
DB Value    4.421053
DB Team     2022 JAC
Name: 94, dtype: object